In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT_v, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()
print("-"*70)

set = gp.Model("Settlement") 

a = set.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = set.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
x = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
My = max(R[i, t, s] for i in range(I) for t in range(T) for s in range(S))
zy = set.addVars(I, T, S, vtype=GRB.BINARY, name="z_y")

d = set.addVars(I, I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")
ep = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_plus")
em = set.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="e_minus")
Me = max(sum(R[i, t, s] for i in range(I)) for t in range(T) for s in range(S))
ze = set.addVars(T, S, vtype=GRB.BINARY, name="z_e") 
set.update()

# obj. func.
obj = gp.quicksum(
    P_DA[t] * a[t] for t in range(T)
) + gp.quicksum(
    1/S * (P_RT_v[t, s] * bp[t, s] - P_PN[t] * bm[t, s])
    for t in range(T) for s in range(S)
)

set.setObjective(obj, GRB.MAXIMIZE)

# x로 y+, y- 결정 지어주기
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(R[i, t, s] - x[i, t, s] == yp[i, t, s] - ym[i, t, s])

# rt 조건
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(yp[i, t, s] <= R[i, t, s])

# alpha, beta+, beta- 결정
for t in range(T):
    for s in range(S):
        set.addConstr(a[t] == gp.quicksum(x[i, t, s] for i in range(I)))
        set.addConstr(bp[t,s] == gp.quicksum(ep[i, t, s] for i in range(I)))
        set.addConstr(bm[t,s] == gp.quicksum(em[i, t, s] for i in range(I)))


# sum e+, sum e- 결정 지어주기
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - gp.quicksum(x[i, t, s] for i in range(I)) 
                    == gp.quicksum(ep[i, t, s] for i in range(I)) - gp.quicksum(em[i, t, s] for i in range(I)))

# sum e+ <= sum R
for t in range(T):
    for s in range(S):
        set.addConstr(gp.quicksum(ep[i, t, s] for i in range(I)) <= gp.quicksum(R[i, t, s] for i in range(I)))

# e+, e- 정의
for i in range(I):
    for t in range(T):
        for s in range(S):
            # e_plus: 초과량에서 준 양을 뺀 것
            set.addConstr(
                ep[i,t,s] == yp[i,t,s] - gp.quicksum(d[i,j,t,s] for j in range(I) if j != i)
            )
            # e_minus: 부족량에서 받은 양을 뺀 것
            set.addConstr(
                em[i,t,s] == ym[i,t,s] - gp.quicksum(d[j,i,t,s] for j in range(I) if j != i)
            )

# 전력 이동 제한
for i in range(I):
    for t in range(T):
        for s in range(S):
            # 본인이 주는 양은 자신의 y+을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[i, j, t, s] for j in range(I) if j != i)
                <= yp[i, t, s]
            )
            # 본인이 받는 양은 자신의 y-을 넘을 수 없음
            set.addConstr(
                gp.quicksum(d[j, i, t, s] for j in range(I) if j != i)
                <= ym[i, t, s]
            )

# 자기 자신과의 거래 방지
for i in range(I):
    for t in range(T):
        for s in range(S):
            set.addConstr(d[i, i, t, s] == 0)

set.optimize()
            
x_v  = np.array([[[set.getVarByName(f"x[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
yp_v = np.array([[[set.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
ym_v = np.array([[[set.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
a_v  = np.array([set.getVarByName(f"alpha[{t}]").X for t in range(T)])
bp_v = np.array([[set.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)])
bm_v = np.array([[set.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)])

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=30
✅ 시뮬레이션 초기화 완료: S=30, Randomness='high', M1=722.00, M2=1957.00
----------------------------------------------------------------------
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D81)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28800 rows, 41784 columns and 124560 nonzeros
Model fingerprint: 0x6f50645f
Variable types: 37464 continuous, 4320 integer (4320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 28440 rows and 40583 columns
Presolve time: 0.04s
Presolved: 360 rows, 1201 columns, 2580 nonzeros
Variable types: 1201 continuous, 0 integer (0 binary)

Root relaxation: objective 1.567562e+06, 80 iterations, 0.00 seconds (0.00 work units)

    Nodes    |

In [67]:
idx = 0
xv   = x_v[:, :, idx]
ypv  = yp_v[:, :, idx]
ymv  = ym_v[:, :, idx]
av   = a_v
bpv  = bp_v[:, idx]
bmv  = bm_v[:, idx]
P_RT = P_RT_v[:, idx]

In [68]:
delta_DA = np.zeros(T)
delta_BMp = np.zeros(T)
delta_BMm = np.zeros(T)
lambd = np.zeros((I, T))
total_surplus = np.zeros(T)

for t in range(T):
    delta_DA[t]  = av[t]  - np.sum(xv[:, t])
    delta_BMp[t] = bpv[t] - np.sum(ypv[:, t])
    delta_BMm[t] = bmv[t] - np.sum(ymv[:, t])
    total_surplus[t]   = P_DA[t] * delta_DA[t] + P_RT[t] * delta_BMp[t] - P_PN[t] * delta_BMm[t]
    
print(f"{'t':>2} | {'ΔDA':>10} | {'ΔBM+':>12} | {'ΔBM-':>12} || {'Surplus':>10}")
print("-" * 60)
for t in range(T):
    print(f"{t:2} | {delta_DA[t]:10.1f} | {delta_BMp[t]:12.1f} | {delta_BMm[t]:12.1f} || {total_surplus[t]:10.1f}")
print("-" * 60)
print(f"{'Σ':>2} | {delta_DA.sum():>10} | {delta_BMp.sum():>12} | {delta_BMm.sum():>12} || {total_surplus.sum():10.1f}")

 t |        ΔDA |         ΔBM+ |         ΔBM- ||    Surplus
------------------------------------------------------------
 0 |        0.0 |          0.0 |          0.0 ||        0.0
 1 |        0.0 |          0.0 |          0.0 ||        0.0
 2 |        0.0 |          0.0 |          0.0 ||        0.0
 3 |        0.0 |          0.0 |          0.0 ||        0.0
 4 |        0.0 |          0.0 |          0.0 ||        0.0
 5 |        0.0 |          0.0 |          0.0 ||        0.0
 6 |        0.0 |         -2.0 |         -2.0 ||      153.9
 7 |        0.0 |         -6.0 |         -6.0 ||      434.7
 8 |        0.0 |        -35.0 |        -35.0 ||     1299.8
 9 |        0.0 |       -143.0 |       -143.0 ||     9875.5
10 |        0.0 |       -299.0 |       -299.0 ||    21712.8
11 |        0.0 |          0.0 |          0.0 ||        0.0
12 |        0.0 |          0.0 |          0.0 ||        0.0
13 |        0.0 |          0.0 |          0.0 ||        0.0
14 |        0.0 |       -821.0 |       

In [69]:
delta_DAp = np.where(delta_DA > 0,  delta_DA, 0)
delta_DAm = np.where(delta_DA < 0, -delta_DA, 0)
# print(f"{'t':>2} | {'α':>6} | {'∑x':>6} | {'ΔDA':>8} | {'ΔDA+':>8} | {'ΔDA-':>8}")
# print("-" * 55)
# for t in range(T):
#     print(f"{t:2} | {av[t]:6.2f} | {np.sum(xv[:,t]):6.2f} | {delta_DA[t]:8.2f} | {delta_DAp[t]:8.2f} | {delta_DAm[t]:8.2f}")

E_INp = np.zeros(T)
E_INm = np.zeros(T)

for t in range(T):
    E_INp[t] = max(np.sum(ypv[:, t]) - (bpv[t] + delta_DAp[t]),0)
    E_INm[t] = max(np.sum(ymv[:, t]) - (bmv[t] + delta_DAm[t]),0)
    
print(f"{'t':>2} | {'∑y⁺':>9} | {'β⁺':>9} | {'ΔDA+':>7} | {'E_IN⁺':>8} || {'∑y⁻':>7} | {'β⁻':>6} | {'ΔDA-':>6} | {'E_IN⁻':>7}")
print("-" * 90)
for t in range(T):
    sum_phi_p = np.sum(ypv[:, t])
    sum_phi_m = np.sum(ymv[:, t])
    print(f"{t:2} | {sum_phi_p:9.2f} | {bpv[t]:9.2f} | {delta_DAp[t]:7.2f} | {E_INp[t]:8.2f} ||"
          f" {sum_phi_m:7.2f} | {bmv[t]:6.2f} | {delta_DAm[t]:6.2f} | {E_INm[t]:7.2f}")

 t |       ∑y⁺ |        β⁺ |    ΔDA+ |    E_IN⁺ ||     ∑y⁻ |     β⁻ |   ΔDA- |   E_IN⁻
------------------------------------------------------------------------------------------
 0 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 1 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 2 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 3 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 4 |      1.00 |      1.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 5 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 6 |      4.00 |      2.00 |    0.00 |     2.00 ||    2.00 |   0.00 |   0.00 |    2.00
 7 |      6.00 |      0.00 |    0.00 |     6.00 ||    7.00 |   1.00 |   0.00 |    6.00
 8 |     35.00 |      0.00 |    0.00 |    35.00 ||   35.00 |   0.00 |   0.00 |   35.00
 9 |    167.00 |     24.00 |    0.00 | 

In [70]:
m = 1e-16
e_INp = np.zeros((I, T))
e_INm = np.zeros((I, T))

for t in range(T):
    for i in range(I):
        e_INp[i, t] = (ypv[i, t] / (np.sum(ypv[:, t]) + m)) * E_INp[t]
        e_INm[i, t] = (ymv[i, t] / (np.sum(ymv[:, t]) + m)) * E_INm[t]

print(f"{'i':>2} {'t':>2} | {'y⁺':>8} | {'∑y⁺':>8} | {'e_IN⁺':>10} || {'y⁻':>8} | {'∑y⁻':>8} | {'e_IN⁻':>10}")
print("-" * 80)
for t in range(8, 22):
    for i in range(I):
        sum_phi_p = np.sum(ypv[:, t])
        sum_phi_m = np.sum(ymv[:, t])
        print(f"{i:2} {t:2} | {ypv[i, t]:8.2f} | {sum_phi_p:8.2f} | {e_INp[i, t]:10.2f} ||"
              f" {ymv[i, t]:8.2f} | {sum_phi_m:8.2f} | {e_INm[i, t]:10.2f}")

 i  t |       y⁺ |      ∑y⁺ |      e_IN⁺ ||       y⁻ |      ∑y⁻ |      e_IN⁻
--------------------------------------------------------------------------------
 0  8 |    12.00 |    35.00 |      12.00 ||     5.50 |    35.00 |       5.50
 1  8 |    12.00 |    35.00 |      12.00 ||     5.50 |    35.00 |       5.50
 2  8 |     5.00 |    35.00 |       5.00 ||    12.00 |    35.00 |      12.00
 3  8 |     5.00 |    35.00 |       5.00 ||    12.00 |    35.00 |      12.00
 4  8 |     1.00 |    35.00 |       1.00 ||     0.00 |    35.00 |       0.00
 0  9 |     3.00 |   167.00 |       2.57 ||   143.00 |   143.00 |     143.00
 1  9 |    83.00 |   167.00 |      71.07 ||     0.00 |   143.00 |       0.00
 2  9 |    45.00 |   167.00 |      38.53 ||     0.00 |   143.00 |       0.00
 3  9 |    29.00 |   167.00 |      24.83 ||     0.00 |   143.00 |       0.00
 4  9 |     7.00 |   167.00 |       5.99 ||     0.00 |   143.00 |       0.00
 0 10 |    55.00 |   586.00 |      28.06 ||   299.00 |   299.00 |     29

In [71]:
delta_INp = np.zeros(T)
delta_INm = np.zeros(T)
Lambda_IN = np.zeros(T)
rho_IN = np.zeros(T)

for t in range(T):
    # if delta_BMp[t] < 0:
    if delta_BMp[t] < 0 and E_INp[t] > 1e-6:
        delta_INp[t] = min(E_INp[t], -delta_BMp[t])
    else:
        delta_INp[t] = 0
    
    # if delta_BMm[t] < 0:
    if delta_BMm[t] < 0 and E_INm[t] > 1e-6:
        delta_INm[t] = min(E_INm[t], -delta_BMm[t])
    else:
        delta_INm[t] = 0
        
    Lambda_IN[t] = (P_RT[t] - P_DA[t]) * delta_INp[t] - (P_PN[t] - P_DA[t]) * delta_INm[t]
    rho_IN[t] = Lambda_IN[t] / (E_INp[t] + E_INm[t] + m)
    
print(f"{'t':>2} | {'ΔBM+':>7} | {'E_IN+':>7} | {'ΔIN+':>7} || "
      f"{'ΔBM-':>7} | {'E_IN-':>7} | {'ΔIN-':>7} || "
      f"{'Λ_IN':>8} | {'ρ_IN':>8}")
print("-" * 90)

for t in range(T):
    print(f"{t:2} | {delta_BMp[t]:7.2f} | {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} || "
          f"{delta_BMm[t]:7.2f} | {E_INm[t]:7.2f} | {delta_INm[t]:7.2f} || "
          f"{Lambda_IN[t]:8.2f} | {rho_IN[t]:8.2f}")

 t |    ΔBM+ |   E_IN+ |    ΔIN+ ||    ΔBM- |   E_IN- |    ΔIN- ||     Λ_IN |     ρ_IN
------------------------------------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||    -0.00 |    -0.00
 1 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||    -0.00 |    -0.00
 2 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 3 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||    -0.00 |    -0.00
 4 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||    -0.00 |    -0.00
 5 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 6 |   -2.00 |    2.00 |    2.00 ||   -2.00 |    2.00 |    2.00 ||  -153.90 |   -38.48
 7 |   -6.00 |    6.00 |    6.00 ||   -6.00 |    6.00 |    6.00 ||  -434.66 |   -36.22
 8 |  -35.00 |   35.00 |   35.00 ||  -35.00 |   35.00 |   35.00 || -1299.76 |   -18.57
 9 | -143.00 |  143.00 |  143.00 || -14

In [72]:
# Lambda_b = np.zeros(T)
# rho_b = np.zeros(T)
# delta_bp = np.zeros(T)
# delta_bm = np.zeros(T)
# e_bp = np.zeros((I, T))
# e_bm = np.zeros((I, T))

# for t in range(T):
#     delta_bp[t] = delta_BMp[t] - delta_INp[t]
#     delta_bm[t] = delta_BMm[t] - delta_INm[t]
#     Lambda_b[t] = (P_RT[t] - P_DA[t]) * delta_bp[t] - (P_PN[t] - P_DA[t]) * delta_bm[t]
    
#     e_bp[:, t] = ypv[:, t] - e_INp[:, t]
#     e_bm[:, t] = ymv[:, t] - e_INm[:, t]
    
#     total_b = np.sum(e_bp[:, t] + e_bm[:, t]+m)

#     # rho_b[t] = Lambda_b[t] / (total_b + m)
#     if total_b < 1e-10:
#         rho_b[t] = 0
#     # Lambda_b[t]는 유지 (반영되지 않음)
#     else:
#         rho_b[t] = Lambda_b[t] / (total_b + m)
    
# print(f"{'i':>2} {'t':>2} | {'y⁺':>7} | {'e_IN⁺':>7} | {'e_b+':>9} || "
#       f"{'y⁻':>7} | {'e_IN⁻':>7} | {'e_b-':>9} || {'Λ_b':>10} | {'ρ_b':>8}")
# print("-" * 120)

# for t in range(8, 22):
#     for i in range(I):
#         print(f"{i:2} {t:2} | "
#               f"{ypv[i, t]:7.4f} | {e_INp[i, t]:7.4f} | {e_bp[i, t]:9.4f} || "
#               f"{ymv[i, t]:7.4f} | {e_INm[i, t]:7.4f} | {e_bm[i, t]:9.4f} || "
#               f"{Lambda_b[t]:10.2f} | {rho_b[t]:8.2f}")

In [73]:
lambda_it = np.zeros((I, T))
lambda_sum = np.zeros(T)

for t in range(T):
    for i in range(I):
        lambda_it[i, t] += rho_IN[t] * (e_INp[i, t] + e_INm[i, t])
        # lambda_it[i, t] += rho_b[t] * (e_bp[i, t] + e_bm[i, t])
    lambda_sum[t] = np.sum(lambda_it[:, t])     

print(f"{'i':>2} {'t':>2} | {'λ_it':>10}")
print("-" * 20)
for t, i in product(range(8, 10), range(I)):
    print(f"{i:2} {t:2} | {lambda_it[i, t]:10.4f}")

 i  t |       λ_it
--------------------
 0  8 |  -324.9395
 1  8 |  -324.9395
 2  8 |  -315.6555
 3  8 |  -315.6555
 4  8 |   -18.5680
 0  9 | -5026.4668
 1  9 | -2454.0985
 2  9 | -1330.5353
 3  9 |  -857.4561
 4  9 |  -206.9722


In [74]:
print(f"{'t':>2} | {'Λ_IN':>8} || {'∑λ_it':>10} | {'Difference':>10}")
print("-" * 55)

total_IN = 0
total_b  = 0
total_lambda = 0

for t in range(T):
#     total_lhs = Lambda_IN[t] + Lambda_b[t]
    total_lhs = Lambda_IN[t]
    diff = total_lhs - lambda_sum[t]

    total_IN     += Lambda_IN[t]
#     total_b      += Lambda_b[t]
    total_lambda += lambda_sum[t]

#     print(f"{t:2} | {Lambda_IN[t]:8.2f} | {Lambda_b[t]:8.2f} || "
#           f"{lambda_sum[t]:10.2f} | {diff:10.2f}")
    print(f"{t:2} | {Lambda_IN[t]:8.2f} || "
          f"{lambda_sum[t]:10.2f} | {diff:10.2f}")
    
print("-" * 55)
# print(f"{'Σ':>2} | {total_IN:8.2f} | {total_b:8.2f}|| "
#       f"{total_lambda:10.2f} | {total_IN + total_b - total_lambda:10.2f}")
print(f"{'Σ':>2} | {total_IN:8.2f}|| "
      f"{total_lambda:10.2f} | {total_IN - total_lambda:10.2f}")
print("-" * 55)
print(f"Confirm: {total_surplus.sum()}")

 t |     Λ_IN ||      ∑λ_it | Difference
-------------------------------------------------------
 0 |    -0.00 ||       0.00 |      -0.00
 1 |    -0.00 ||       0.00 |      -0.00
 2 |     0.00 ||       0.00 |       0.00
 3 |    -0.00 ||       0.00 |      -0.00
 4 |    -0.00 ||       0.00 |      -0.00
 5 |     0.00 ||       0.00 |       0.00
 6 |  -153.90 ||    -153.90 |       0.00
 7 |  -434.66 ||    -434.66 |       0.00
 8 | -1299.76 ||   -1299.76 |      -0.00
 9 | -9875.53 ||   -9875.53 |       0.00
10 | -21712.84 ||  -21712.84 |      -0.00
11 |     0.00 ||       0.00 |       0.00
12 |     0.00 ||       0.00 |       0.00
13 |     0.00 ||       0.00 |       0.00
14 | -86730.23 ||  -86730.23 |       0.00
15 |     0.00 ||       0.00 |       0.00
16 | -72083.89 ||  -72083.89 |       0.00
17 |     0.00 ||       0.00 |       0.00
18 |     0.00 ||       0.00 |       0.00
19 | -26682.77 ||  -26682.77 |       0.00
20 | -7132.52 ||   -7132.52 |      -0.00
21 |  -514.04 ||    -514.04 |       0.

In [75]:
print(f"{'t':>2} || {'E_IN+':>7} | {'ΔIN⁺':>7} | {'Δb⁺':>7} | {'ΔBM⁺':>7} | {'ΔDA+':>7} || {'E_IN-':>7} | {'ΔIN⁻':>7} | {'Δb⁻':>7} | {'ΔBM⁻':>7} | {'ΔDA-':>7}")
print("-" * 105)
for t in range(T):
    print(f"{t:2} || {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} | {delta_bp[t]:7.2f} | {delta_BMp[t]:7.2f} | {delta_DAp[t]:7.2f} || "
          f"{E_INm[t]:7.2f} | {delta_INm[t]:7.2f} | {delta_bm[t]:7.2f} | {delta_BMm[t]:7.2f} | {delta_DAm[t]:7.2f} ")

 t ||   E_IN+ |    ΔIN⁺ |     Δb⁺ |    ΔBM⁺ |    ΔDA+ ||   E_IN- |    ΔIN⁻ |     Δb⁻ |    ΔBM⁻ |    ΔDA-
---------------------------------------------------------------------------------------------------------
 0 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 1 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 2 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 3 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 4 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 5 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 |    0.00 |    0.00 
 6 ||    2.00 |    2.00 |   -4.00 |   -2.00 |    0.00 ||    2.00 |    2.00 |   -4.00 |   -2.00 |    0.00 
 7 ||    6.00 |    6.00 |  -12.00 |   -6.00 |  

In [76]:
print(f"{'t':>2} | {'ΔDA+':>7} | {'ΔDA-':>7} | {'ΔBM+':>7} | {'ΔBM-':>7} | {'Δb⁺':>7} | {'Δb⁻':>7}")
print("-" * 65)
for t in range(T):
    print(f"{t:2} | {delta_DAp[t]:7.2f} | {delta_DAm[t]:7.2f} | {delta_BMp[t]:7.2f} | {delta_BMm[t]:7.2f} | "
          f"{delta_bp[t]:7.2f} | {delta_bm[t]:7.2f}")

 t |    ΔDA+ |    ΔDA- |    ΔBM+ |    ΔBM- |     Δb⁺ |     Δb⁻
-----------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 1 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 2 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 3 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 4 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 5 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
 6 |    0.00 |    0.00 |   -2.00 |   -2.00 |   -4.00 |   -4.00
 7 |    0.00 |    0.00 |   -6.00 |   -6.00 |  -12.00 |  -12.00
 8 |    0.00 |    0.00 |  -35.00 |  -35.00 |  -70.00 |  -70.00
 9 |    0.00 |    0.00 | -143.00 | -143.00 | -286.00 | -286.00
10 |    0.00 |    0.00 | -299.00 | -299.00 | -598.00 | -598.00
11 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
12 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00
13 |    0.00 |    0.00 |    0.00 |    0.00 |    0.00